### Despliegue del Modelo en Databricks con MLflow

Este notebook forma parte del trabajo final del curso de MLOps  y se centra en la fase de **despliegue** del modelo de predicción de accidentes en minería.

El objetivo es cargar el modelo previamente entrenado y registrado en **MLflow**, para luego realizar pruebas de inferencia antes de su integración en un entorno productivo.  
El flujo de trabajo incluye:

- **Carga del modelo desde MLflow Model Registry**, utilizando una versión específica o la última versión en **Production**.
- **Ejecución de inferencias locales** sobre datos de prueba, validando la correcta aplicación del modelo antes del despliegue definitivo.

Este proceso es fundamental en **MLOps**, ya que permite asegurar que el modelo puede realizar predicciones de manera confiable antes de ser utilizado en entornos de producción.

<style>
    p { text-align: justify; }
</style>


### 1. Importación de librerías y configuración del entorno
Se importan las librerías necesarias para la ejecución del notebook, incluyendo MLflow para la gestión del modelo y Apache Spark para la manipulación de datos. Se inicia la sesión de Spark y se muestra un mensaje de inicio

In [0]:
# Paso 1: Importar librerías
import mlflow
import mlflow.spark
import pandas as pd

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print("Notebook 04: Despliegue - Iniciando...")


Notebook 04: Despliegue - Iniciando...


### 2. Carga del modelo registrado en MLflow
Se carga el modelo registrado en MLflow Model Registry para realizar predicciones en un entorno de despliegue. Dependiendo del estado del modelo en MLflow, se puede cargar una versión específica o la última promovida a Production.

In [0]:
# Paso 2: Cargar el modelo registrado desde MLflow
model_uri = "models:/AccidentPrediction2025/1"

# Cargamos la pipeline Spark
pipeline_model = mlflow.spark.load_model(model_uri)

print(f"Modelo cargado desde URI: {model_uri}")


2025/03/04 13:26:44 INFO mlflow.spark: 'models:/AccidentPrediction2025/1' resolved as 'abfss://unity-catalog-storage@dbstorageyp3hv5lr3xzh6.dfs.core.windows.net/1781258311325241/models/f0b5bfa6-b545-4a3a-8f96-eddccdc04e61/versions/74acc5b5-5c4b-414d-b811-03fa453d36b7'


Modelo cargado desde URI: models:/AccidentPrediction2025/1


### 3. Inferencia local con Spark DataFrame
Se prueba el modelo cargado en un conjunto de datos de prueba construido manualmente. El objetivo es validar que el modelo puede realizar predicciones correctamente antes de desplegarlo en un entorno real.

In [0]:
# Paso 3: Probar inferencia local con Spark DataFrame

# Construimos un dataframe Spark con columnas:
# Shift_Type, Weather_Risk_Index, Job_Risk_Level, Hours_Worked,
# Employee_Experience_Years, Safety_Violations, Inspection_Frequency, Temperature_C

data_sample = [
    (1.0, 3.5, 2.0, 8.0, 5.0, 0.0, 3.0, 25.0),
    (2.0, 1.5, 1.0, 10.0, 2.0, 1.0, 2.0, 15.0),
    (3.0, 4.5, 3.0, 6.0,  8.0, 2.0, 4.0, 20.0)
]

columns_list = [
    "Shift_Type", 
    "Weather_Risk_Index", 
    "Job_Risk_Level", 
    "Hours_Worked",
    "Employee_Experience_Years", 
    "Safety_Violations", 
    "Inspection_Frequency", 
    "Temperature_C"
]

df_test_local = spark.createDataFrame(data_sample, columns_list)

print("DataFrame de prueba local:")
display(df_test_local)

# Usar pipeline_model.transform() para hacer inferencia
df_pred = pipeline_model.transform(df_test_local)

print("Predicciones:")
display(df_pred)


DataFrame de prueba local:


Shift_Type,Weather_Risk_Index,Job_Risk_Level,Hours_Worked,Employee_Experience_Years,Safety_Violations,Inspection_Frequency,Temperature_C
1.0,3.5,2.0,8.0,5.0,0.0,3.0,25.0
2.0,1.5,1.0,10.0,2.0,1.0,2.0,15.0
3.0,4.5,3.0,6.0,8.0,2.0,4.0,20.0


Predicciones:


Shift_Type,Weather_Risk_Index,Job_Risk_Level,Hours_Worked,Employee_Experience_Years,Safety_Violations,Inspection_Frequency,Temperature_C,rawFeatures,scaledFeatures,rawPrediction,probability,prediction
1.0,3.5,2.0,8.0,5.0,0.0,3.0,25.0,"Map(vectorType -> dense, length -> 8, values -> List(1.0, 3.5, 2.0, 8.0, 5.0, 0.0, 3.0, 25.0))","Map(vectorType -> dense, length -> 8, values -> List(2.1993108575882223, 12.100088611306038, 2.9664884030516085, 4.706502809339085, 0.5792997150322952, 0.0, 0.953370798972236, 2.8949151637859534))","Map(vectorType -> dense, length -> 2, values -> List(0.0, 50.0))","Map(vectorType -> dense, length -> 2, values -> List(0.0, 1.0))",1.0
2.0,1.5,1.0,10.0,2.0,1.0,2.0,15.0,"Map(vectorType -> dense, length -> 8, values -> List(2.0, 1.5, 1.0, 10.0, 2.0, 1.0, 2.0, 15.0))","Map(vectorType -> dense, length -> 8, values -> List(4.398621715176445, 5.185752261988302, 1.4832442015258043, 5.883128511673856, 0.23171988601291807, 0.34762533202385265, 0.6355805326481573, 1.736949098271572))","Map(vectorType -> dense, length -> 2, values -> List(0.0, 50.0))","Map(vectorType -> dense, length -> 2, values -> List(0.0, 1.0))",1.0
3.0,4.5,3.0,6.0,8.0,2.0,4.0,20.0,"Map(vectorType -> dense, length -> 8, values -> List(3.0, 4.5, 3.0, 6.0, 8.0, 2.0, 4.0, 20.0))","Map(vectorType -> dense, length -> 8, values -> List(6.597932572764667, 15.557256785964906, 4.449732604577413, 3.529877107004314, 0.9268795440516723, 0.6952506640477053, 1.2711610652963146, 2.315932131028763))","Map(vectorType -> dense, length -> 2, values -> List(0.0, 50.0))","Map(vectorType -> dense, length -> 2, values -> List(0.0, 1.0))",1.0
